# LightFM (user item trancation 2)
* user: 各ユーザーの最新購入の直近user_records個のデータのみ学習に使う
* item: 学習期間の最後item_days日に含まれるアイテムのみ学習＆予測に使う

In [1]:
import datetime

import faiss
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import psutil
from lightfm import LightFM
from scipy.sparse import lil_matrix
from utils import train_valid_split

import schema
from metric import mapk

In [2]:
transactions = pd.read_pickle('input/transformed/transactions_train.pkl')[['t_dat', 'user', 'item']]
users = pd.read_pickle('input/transformed/users.pkl')['user']
items = pd.read_pickle('input/transformed/items.pkl')['item']
TOPK = 12

valid_start_date = datetime.date(2020, 9, 16)
transactions_train, transactions_valid = train_valid_split(transactions, valid_start_date, 70)

[I 220309 11:51:59 utils:29] train: [2020-07-08, 2020-09-16)
[I 220309 11:51:59 utils:31] # of records: 2809233
[I 220309 11:51:59 utils:16] valid: [2020-09-16, 2020-09-23)
[I 220309 11:52:00 utils:18] # of records: 240311


In [18]:
def objective(trial: optuna.Trial) -> float:
    user_records = trial.suggest_int('user_records', 1, 100)
    item_days = trial.suggest_int('item_days', 7, 70, 7)
    no_components = trial.suggest_int('no_components', 128, 1024, 128)
    epochs = 100

    lightfm_params = {
        'no_components': no_components,
        'learning_schedule': 'adadelta',
        'loss': 'bpr',
        'learning_rate': 0.005,
        'item_alpha': 1e-8,
        'user_alpha': 1e-8,
    }

    start_date = valid_start_date - datetime.timedelta(days=item_days)
    transactions_item = sorted(transactions_train.query("t_dat >= @start_date")['item'].unique())

    print('item days', item_days)
    print(len(transactions_item))
    mp_item = dict(zip(transactions_item, range(len(transactions_item))))
    tr_train = transactions_train.copy()

    tr_train = tr_train.query("item in @transactions_item").reset_index(drop=True)
    tr_train = tr_train.drop_duplicates(ignore_index=True)

    tr_train['rnk'] = tr_train.groupby('user')['t_dat'].rank(method='first')
    print('user records', user_records)
    print('before', len(tr_train))
    tr_train = tr_train.query("rnk <= @user_records").reset_index(drop=True)
    print('after', len(tr_train))

    tr_valid = transactions_valid.copy()
    tr_train['item'] = tr_train['item'].apply(lambda x: mp_item[x])
    tr_valid['item'] = tr_valid['item'].apply(lambda x: mp_item.get(x, 123456789))
    val = tr_valid.groupby('user')['item'].apply(list).reset_index()
    n_item = len(transactions_item)
    n_user = len(users)

    train = lil_matrix((n_user, n_item))
    train[tr_train.user, tr_train.item] = 1

    model = LightFM(**lightfm_params)
    model.fit(train, epochs=epochs, num_threads=psutil.cpu_count(logical=False), verbose=True)

    index = faiss.index_factory(no_components, "Flat", faiss.METRIC_INNER_PRODUCT)
    index = faiss.index_cpu_to_gpu(faiss.StandardGpuResources(), 0, index)
    index.add(model.item_embeddings)
    _, idxs = index.search(model.user_embeddings, TOPK)

    return mapk(val.item, idxs[val.user])

In [19]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=3600*8)

[I 2022-03-09 01:01:25,758] A new study created in memory with name: no-name-db7597cb-dc63-423d-9f6f-36b3089adc0a


item days 28
29009
user days 7
before 2775154
after 1688381


Epoch: 100%|██████████| 100/100 [04:51<00:00,  2.92s/it]
[I 2022-03-09 01:06:37,291] Trial 0 finished with value: 0.02255802923842525 and parameters: {'user_days': 7, 'item_days': 28, 'no_components': 640}. Best is trial 0 with value: 0.02255802923842525.


item days 42
32792
user days 56
before 2797986
after 2718059


Epoch: 100%|██████████| 100/100 [04:35<00:00,  2.76s/it]
[I 2022-03-09 01:11:28,280] Trial 1 finished with value: 0.01789910280402429 and parameters: {'user_days': 56, 'item_days': 42, 'no_components': 384}. Best is trial 0 with value: 0.02255802923842525.


...

Epoch: 100%|██████████| 100/100 [08:23<00:00,  5.04s/it]
[I 2022-03-09 08:51:49,086] Trial 63 finished with value: 0.022403361455568465 and parameters: {'user_days': 14, 'item_days': 21, 'no_components': 1024}. Best is trial 59 with value: 0.0231487379063096.


item days 35
30979
user days 7
before 2789764
after 1694346


Epoch: 100%|██████████| 100/100 [06:36<00:00,  3.97s/it]
[I 2022-03-09 08:58:52,353] Trial 64 finished with value: 0.022961003423987963 and parameters: {'user_days': 7, 'item_days': 35, 'no_components': 896}. Best is trial 59 with value: 0.0231487379063096.


item days 35
30979
user days 21
before 2789764
after 2053176


Epoch: 100%|██████████| 100/100 [05:42<00:00,  3.42s/it]
[I 2022-03-09 09:04:55,228] Trial 65 finished with value: 0.021316396892195878 and parameters: {'user_days': 21, 'item_days': 35, 'no_components': 640}. Best is trial 59 with value: 0.0231487379063096.


In [20]:
study.trials_dataframe().sort_values(by='value', ascending=False).head(20)

,number,value,datetime_start,datetime_complete,duration,params_item_days,params_no_components,params_user_days,state
59,59,0.023149,2022-03-09 08:14:19.321905,2022-03-09 08:22:35.091491,0 days 00:08:15.769586,28,1024,7,COMPLETE
36,36,0.023028,2022-03-09 05:07:33.780202,2022-03-09 05:15:49.789598,0 days 00:08:16.009396,21,1024,7,COMPLETE
52,52,0.022999,2022-03-09 07:15:42.262143,2022-03-09 07:23:23.162689,0 days 00:07:40.900546,14,1024,7,COMPLETE
49,49,0.022996,2022-03-09 06:49:08.895512,2022-03-09 06:57:28.581912,0 days 00:08:19.686400,28,1024,7,COMPLETE
45,45,0.022981,2022-03-09 06:19:30.756071,2022-03-09 06:27:51.839865,0 days 00:08:21.083794,56,1024,7,COMPLETE
56,56,0.022971,2022-03-09 07:46:00.587434,2022-03-09 07:54:22.355102,0 days 00:08:21.767668,35,1024,7,COMPLETE
64,64,0.022961,2022-03-09 08:51:49.087369,2022-03-09 08:58:52.353776,0 days 00:07:03.266407,35,896,7,COMPLETE
29,29,0.022936,2022-03-09 04:17:41.974663,2022-03-09 04:25:23.328200,0 days 00:07:41.353537,14,1024,7,COMPLETE
31,31,0.022914,2022-03-09 04:33:41.703001,2022-03-09 04:40:30.519930,0 days 00:06:48.816929,14,896,7,COMPLETE
21,21,0.022893,2022-03-09 03:17:52.042700,2022-03-09 03:26:16.200555,0 days 00:08:24.157855,49,1024,7,COMPLETE


In [22]:
len(transactions_train.user.unique())

439368

In [25]:
len(transactions_train.query("t_dat > '2020-08-01'").user.unique())

329034

In [27]:
len(users) - len(transactions_train.query("t_dat > '2020-08-01'").user.unique())

1042946

In [29]:
v = set(transactions_valid.user)

In [36]:
t = set(transactions_train.query("t_dat > '2020-08-25'").user)
print(len(v - t))

42230


In [37]:
t = set(transactions_train.user)
print(len(v - t))

23068
